# Goal: Song Popularity Prediction using non-linear models
## Name: Ashwani Rajan
### Machine Learning Algo: Decision Tree and Random Forest

In [0]:
from pyspark import SparkContext
from pyspark.sql.window import Window
from pyspark.sql import SparkSession 
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer

from pyspark.sql import SparkSession
from pyspark.sql.types import *

ss = SparkSession.builder.getOrCreate()
sc = ss.sparkContext

In [0]:
spark

Out[2]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 spark://10.215.239.171:7077 
 AppName 
 Databricks Shell

## Reading Data

In [0]:
database = 'oms'
collection = 'song_popularity_data'
user_name = 'oms'
password = 'oms'
address = 'oms-cluster.0navm.mongodb.net'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"

In [0]:
df_oms = spark.read.format("mongo").option("uri",connection_string).load()

In [0]:
%%time
df_oms.show(4)

+--------------------+------------------+-----------+---------+---+--------+----+-------------------+------------------+-----------------+-------+--------------+--------------------+-----+
 _id| artist_id|artist_name| duration|key|loudness|mode| song_hotttnesss| song_id|start_of_fade_out| tempo|time_signature| title| year|
+--------------------+------------------+-----------+---------+---+--------+----+-------------------+------------------+-----------------+-------+--------------+--------------------+-----+
{622bea71bef0293c...|AR9CHCU1187B9A2317| Manchild|331.49342| 11| -7.812| 0| null|SOQJSEV12AB017BD67| 331.493|127.439| 4|Let Me Tell You S...|1960s|
{622bea71bef0293c...|ARUDSNG11F4C83FE36| 17|227.05587| 7| -8.459| 1| null|SODMQWC12A8C131E05| 213.496|167.907| 4| What's Happin|1960s|
{622bea71bef0293c...|ARINJOL1187FB3FE42| Methadrone|384.10404| 11| -10.185| 0|0.39200877147130697|SOUDYLM12AB018C348| 372.036|140.353| 4| Flight to Nowhere|1960s|
{622bea71bef0293c...|AR27YUB1187B9A0939| Jahcoozi|292.46649| 7| -4.722| 1| null|SOVOKHE12AB0189C08| 282.308| 250.02| 3| Namedropper|1960s|
+--------------------+------------------+-----------+---------+---+--------+----+-------------------+------------------+-----------------+-------+--------------+--------------------+-----+
only showing top 4 rows

CPU times: user 28.2 ms, sys: 12.8 ms, total: 41 ms
Wall time: 11.5 s

CPU times: user 28.2 ms, sys: 12.8 ms, total: 41 ms
Wall time: 11.5 s

### Train Test Split
Take all non null values as train and do prediction on the values where "song_hotttnesss" is Null

In [0]:
df_test = df_oms.filter(df_oms['song_hotttnesss'].isNull())
df_train = df_oms.filter(df_oms['song_hotttnesss'].isNotNull())

In [0]:
%%time
df_test.count()
df_train.count()

CPU times: user 469 µs, sys: 68 µs, total: 537 µs
Wall time: 500 ms
Out[20]: 581965

CPU times: user 30 ms, sys: 11.2 ms, total: 41.2 ms
Wall time: 11.7 s

In [0]:
### Select needed columns
df_train = df_train.select('duration','start_of_fade_out', 'time_signature','tempo','key','mode','year','loudness','song_hotttnesss')
df_test = df_test.select('duration','start_of_fade_out', 'time_signature','tempo','key','mode','year','loudness','song_hotttnesss')
df_test.cache()
df_train.cache()

Out[21]: DataFrame[duration: double, start_of_fade_out: double, time_signature: bigint, tempo: double, key: bigint, mode: bigint, year: string, loudness: double, song_hotttnesss: double]

### train valdiation split

In [0]:
data_split = df_train.randomSplit([0.8,0.2], seed = 10)

train_data = data_split[0]
val_data = data_split[1]

train_data.cache()
val_data.cache()

Out[22]: DataFrame[duration: double, start_of_fade_out: double, time_signature: bigint, tempo: double, key: bigint, mode: bigint, year: string, loudness: double, song_hotttnesss: double]

In [0]:
train_data.display()

duration,start_of_fade_out,time_signature,tempo,key,mode,year,loudness,song_hotttnesss
33.41016,33.41,3,94.111,9,0,1960s,-14.766,0.0
44.93016,44.93,1,120.177,2,1,2000s,-16.527,0.0
53.15873,49.319,1,188.972,7,0,1960s,-2.751,0.4191258929591558
53.28934,50.614,4,123.068,0,1,2000s,-10.458,0.3603705827121406
61.67465,61.675,4,124.363,10,0,1960s,-7.409,0.0
61.962,61.962,4,79.293,0,0,2000s,-11.74,0.7118776226158788
63.03302,63.033,4,122.795,10,1,1960s,-15.395,0.450992311636255
74.91873,71.686,3,160.805,3,1,1990s,-23.334,0.5946422445649223
77.19138,77.191,4,73.749,8,1,1960s,-9.435,0.3943948561005826
77.89669,77.897,4,191.936,1,1,2000s,-4.773,0.6266128026956008


## String-Indexing

only year column needs string indexing

In [0]:
si = StringIndexer(inputCol='year', outputCol="year-num")
sm = si.fit(train_data)
train_data_num = sm.transform(train_data).drop('year')
valid_data_num = sm.transform(val_data).drop('year')
df_test_num = sm.transform(df_test).drop('year')

train_data_num = train_data_num.withColumnRenamed("year-num", "year").cache()
valid_data_num = valid_data_num.withColumnRenamed("year-num", "year").cache()
df_test_num = df_test_num.withColumnRenamed("year-num", "year").cache()

In [0]:
%%time
train_data_num.display()

duration,start_of_fade_out,time_signature,tempo,key,mode,loudness,song_hotttnesss,year
33.41016,33.41,3,94.111,9,0,-14.766,0.0,0.0
44.93016,44.93,1,120.177,2,1,-16.527,0.0,1.0
53.15873,49.319,1,188.972,7,0,-2.751,0.4191258929591558,0.0
53.28934,50.614,4,123.068,0,1,-10.458,0.3603705827121406,1.0
61.67465,61.675,4,124.363,10,0,-7.409,0.0,0.0
61.962,61.962,4,79.293,0,0,-11.74,0.7118776226158788,1.0
63.03302,63.033,4,122.795,10,1,-15.395,0.450992311636255,0.0
74.91873,71.686,3,160.805,3,1,-23.334,0.5946422445649223,2.0
77.19138,77.191,4,73.749,8,1,-9.435,0.3943948561005826,0.0
77.89669,77.897,4,191.936,1,1,-4.773,0.6266128026956008,1.0


CPU times: user 480 µs, sys: 54 µs, total: 534 µs
Wall time: 620 µs

CPU times: user 480 µs, sys: 54 µs, total: 534 µs
Wall time: 620 µs

In [0]:
train_data_num.groupBy('year').count().show()

+----+------+
year| count|
+----+------+
 8.0| 27|
 0.0|192830|
 7.0| 56|
 1.0|171169|
 4.0| 10756|
 3.0| 20038|
 2.0| 64151|
 6.0| 663|
 5.0| 5920|
 9.0| 19|
+----+------+

## Vector Assembler

In [0]:
from pyspark.ml.feature import VectorAssembler
cols = ['duration', 'start_of_fade_out', 'time_signature', 'tempo', 'key', 'mode', 'loudness', 'year']
va = VectorAssembler(outputCol="features", inputCols=cols) 
train_data_vec = va.transform(train_data_num).select("features", "song_hotttnesss").withColumnRenamed('song_hotttnesss', 'label')
valid_data_vec = va.transform(valid_data_num).select("features", "song_hotttnesss").withColumnRenamed('song_hotttnesss', 'label')
test_data_vec = va.transform(df_test_num).select("features", "song_hotttnesss").withColumnRenamed('song_hotttnesss', 'label')

In [0]:
%%time
train_data_vec.display()

features,label
"Map(vectorType -> dense, length -> 8, values -> List(33.41016, 33.41, 3.0, 94.111, 9.0, 0.0, -14.766, 0.0))",0.0
"Map(vectorType -> dense, length -> 8, values -> List(44.93016, 44.93, 1.0, 120.177, 2.0, 1.0, -16.527, 1.0))",0.0
"Map(vectorType -> dense, length -> 8, values -> List(53.15873, 49.319, 1.0, 188.972, 7.0, 0.0, -2.751, 0.0))",0.4191258929591558
"Map(vectorType -> dense, length -> 8, values -> List(53.28934, 50.614, 4.0, 123.068, 0.0, 1.0, -10.458, 1.0))",0.3603705827121406
"Map(vectorType -> dense, length -> 8, values -> List(61.67465, 61.675, 4.0, 124.363, 10.0, 0.0, -7.409, 0.0))",0.0
"Map(vectorType -> dense, length -> 8, values -> List(61.962, 61.962, 4.0, 79.293, 0.0, 0.0, -11.74, 1.0))",0.7118776226158788
"Map(vectorType -> dense, length -> 8, values -> List(63.03302, 63.033, 4.0, 122.795, 10.0, 1.0, -15.395, 0.0))",0.450992311636255
"Map(vectorType -> dense, length -> 8, values -> List(74.91873, 71.686, 3.0, 160.805, 3.0, 1.0, -23.334, 2.0))",0.5946422445649223
"Map(vectorType -> dense, length -> 8, values -> List(77.19138, 77.191, 4.0, 73.749, 8.0, 1.0, -9.435, 0.0))",0.3943948561005826
"Map(vectorType -> dense, length -> 8, values -> List(77.89669, 77.897, 4.0, 191.936, 1.0, 1.0, -4.773, 1.0))",0.6266128026956008


CPU times: user 503 µs, sys: 56 µs, total: 559 µs
Wall time: 637 µs

## Decision Tree Regressor with cross-validation

maxDepth: [5,10,15,20,25,30] increasing depth allows more deep tree, i.e. more bushy trees\
maxBins: [16,32,48] increasing maxBins allow more finer splits

Evaluation Metric: RMSE

In [0]:

from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator


evaluator = RegressionEvaluator().setLabelCol("label")\
                .setPredictionCol("prediction")\
                .setMetricName('rmse')

dt = DecisionTreeRegressor()
paramGrid = ParamGridBuilder().addGrid(dt.maxDepth, [5,10,15,20,25,30]).addGrid(dt.maxBins, [16,32,48]).build()

cv = CrossValidator(estimator=dt, 
                      evaluator=evaluator, 
                      numFolds=3, 
                      estimatorParamMaps=paramGrid)






In [0]:
%%time
cvmodel = cv.fit(train_data_vec)

CPU times: user 5.42 s, sys: 1.81 s, total: 7.23 s
Wall time: 21min 45s

In [0]:
%%time
dtpredicts = cvmodel.bestModel.transform(valid_data_vec)

print("Best Model Max depth : %s" % cvmodel.bestModel.getMaxDepth())
print("Best Model Max bins : %s" % cvmodel.bestModel.getMaxBins())
print("RMSE : %s" % evaluator.evaluate(dtpredicts))

Best Model Max depth : 5
Best Model Max bins : 32
RMSE : 0.21282878467532604
CPU times: user 5.88 ms, sys: 5.46 ms, total: 11.3 ms
Wall time: 224 ms

## Random Forest Model

In [0]:
from pyspark.ml.regression import RandomForestRegressor

In [0]:

%%time
rf = RandomForestRegressor(seed = 2, maxDepth=20)
rf_model = rf.fit(train_data_vec)
print(rf_model.toDebugString)

RandomForestRegressionModel: uid=RandomForestRegressor_8eb7fba3f338, numTrees=20, numFeatures=8
 Tree 0 (weight 1.0):
 If (feature 6 <= -7.447)
 If (feature 6 <= -12.1145)
 If (feature 7 in {0.0,9.0})
 If (feature 0 <= 224.35219999999998)
 If (feature 3 <= 86.488)
 If (feature 0 <= 145.815055)
 If (feature 6 <= -13.441)
 If (feature 0 <= 60.890975)
 If (feature 4 <= 1.5)
 If (feature 6 <= -20.977)
 If (feature 4 <= 0.5)
 If (feature 1 <= 58.5375)
 If (feature 3 <= 60.786)
 If (feature 2 <= 2.0)
 Predict: 0.029661687363948728
 Else (feature 2 > 2.0)
 Predict: 0.14869739588433167
 Else (feature 3 > 60.786)
 If (feature 5 <= 0.5)
 Predict: 0.0
 Else (feature 5 > 0.5)
 Predict: 0.2516189195979322
 Else (feature 1 > 58.5375)
 Predict: 0.30416950343527516
 Else (feature 4 > 0.5)
 If (feature 5 <= 0.5)
 Predict: 0.3135621142479342
 Else (feature 5 > 0.5)
 Predict: 0.023216166743418826
 Else (feature 6 > -20.977)
 If (feature 5 <= 0.5)
 If (feature 2 <= 4.5)
 If (feature 4 <= 0.5)
 Predict: 0.27938059329926845
 Else (feature 4 > 0.5)
 If (feature 3 <= 60.786)
 Predict: 0.04781919829947588
 Else (feature 3 > 60.786)
 Predict: 0.0
 Else (feature 2 > 4.5)
 Predict: 0.6464840667661809
 Else (feature 5 > 0.5)
 If (feature 4 <= 0.5)
 If (feature 6 <= -16.3965)
 If (feature 1 <= 58.5375)
 Predict: 0.053001014755887886
 Else (feature 1 > 58.5375)
 Predict: 0.0
 Else (feature 6 > -16.3965)
 If (feature 2 <= 4.5)
 If (feature 2 <= 2.0)
 If (feature 3 <= 60.786)
 Predict: 0.34580223396532944
 Else (feature 3 > 60.786)
 Predict: 0.0
 Else (feature 2 > 2.0)
 If (feature 3 <= 60.786)
 Predict: 0.4348380902608562
 Else (feature 3 > 60.786)
 If (feature 3 <= 82.761)
 Predict: 0.1620293496069234
 Else (feature 3 > 82.761)
 Predict: 0.21204540548371908
 Else (feature 2 > 4.5)
 Predict: 0.0
 Else (feature 4 > 0.5)
 If (feature 2 <= 4.5)
 If (feature 6 <= -15.195)
 If (feature 6 <= -18.057)
 If (feature 3 <= 60.786)
 Predict: 0.12343254722644069
 Else (feature 3 > 60.786)
 Predict: 0.6226207266500394
 Else (feature 6 > -18.057)
 If (feature 3 <= 74.40350000000001)
 Predict: 0.08319831873924796
 Else (feature 3 > 74.40350000000001)
 Predict: 0.2707759989463275
 Else (feature 6 > -15.195)
 If (feature 3 <= 60.786)
 Predict: 0.15323223287422955
 Else (feature 3 > 60.786)
 If (feature 2 <= 2.0)
 Predict: 0.40003965695463956
 Else (feature 2 > 2.0)
 Predict: 0.37253003936607965
 Else (feature 2 > 4.5)
 Predict: 0.0
 Else (feature 4 > 1.5)
 If (feature 1 <= 58.5375)
 If (feature 5 <= 0.5)
 If (feature 6 <= -20.977)
 If (feature 4 <= 4.5)
 If (feature 4 <= 2.5)
 If (feature 3 <= 60.786)
 Predict: 0.2027196992596418
 Else (feature 3 > 60.786)
 If (feature 3 <= 74.40350000000001)
 Predict: 0.0
 Else (feature 3 > 74.40350000000001)
 Predict: 0.21508031850922793
 Else (feature 4 > 2.5)
 Predict: 0.0
 Else (feature 4 > 4.5)
 If (feature 2 <= 2.0)
 Predict: 0.1559377508470846
 Else (feature 2 > 2.0)
 If (feature 4 <= 5.5)
 Predict: 0.0
 Else (feature 4 > 5.5)
 If (feature 4 <= 6.5)
 Predict: 0.39200877147130697
 Else (feature 4 > 6.5)
 If (feature 4 <= 8.5)
 If (feature 2 <= 3.5)
 Predict: 0.26695518627553855
 Else (feature 2 > 3.5)
 Predict: 0.0
 Else (feature 4 > 8.5)
 If (feature 3 <= 60.786)
 Predict: 0.10754015925461397
 Else (feature 3 > 60.786)
 Predict: 0.35134269333499296
 Else (feature 6 > -20.977)
 If (feature 3 <= 82.761)
 Predict: 0.2774584142542839
 Else (feature 3 > 82.761)
 If (feature 2 <= 3.5)
 Predict: 0.0
 Else (feature 2 > 3.5)
 Predict: 0.08898506209184619
 Else (feature 5 > 0.5)
 If (feature 2 <= 6.0)
 If (feature 6 <= -15.195)
 If (feature 3 <= 82.761)
 If (feature 4 <= 6.5)
 If (feature 3 <= 74.40350000000001)
 If (feature 6 <= -20.977)
 If (feature 2 <= 2.0)
 Predict: 0.1543422586486227
 Else (feature 2 > 2.0)
 If (feature 4 <= 2.5)
 If (feature 3 <= 60.786)
 Predict: 0.26133918098087133
 Else (feature 3 > 60.786)
 Predict: 0.0
 Else (feature 4 > 2.5)
 Predict: 0.03347065490692206
 Else (feature 6 > -20.977)
 If (feature 6 <= -16.3965)
 If (

RF Model time :5.14 minutes

In [0]:
rfpredicts = rf_model.transform(valid_data_vec)
print("RMSE : %s" % evaluator.evaluate(rfpredicts))

RMSE : 0.2162198120977653

In [0]:
rf_model.getNumTrees

Out[58]: 20

Number of Trees: 20, Max_depth:20